# First write to DB

In [1]:
import polars as pl
import re 
import time
def view_string(long_string, chunk_size=100):     
    return [long_string[i:i+chunk_size] for i in range(0, len(long_string), chunk_size)]


In [2]:
%cd /home/sebacastillo/willow

/home/sebacastillo/willow


In [3]:
news = pl.read_csv(   
    'output/news.csv',
    dtypes={"content_hash": pl.UInt64},     
)
newsner = pl.read_csv(   
    'output/newsner.csv',
    dtypes={"content_hash": pl.UInt64},     
)

In [4]:
news.head(1)

index,topic,date_extract,date_article,content,portal,link,link_sim_score,title,summary,summary_llm,summary_sim_score,authors,state,city,content_hash,content_nchar
i64,str,str,str,str,str,str,f64,str,str,str,f64,str,str,str,u64,i64
1,"""narcotráfico""","""2023-08-16""","""2023-08-16""","""Seguinos Por L…","""https://www.ar…","""https://www.la…",0.561929,"""Condenaron a s…","""Brian está det…","""Seis personas …",0.5241,"""n-a""","""Santa Fe""","""Rosario""",9259959111029284149,4543


In [5]:
news = news.with_columns(
    pl.concat_str(
        [
            pl.col('state'),
            pl.col('city'),
            pl.col("title"),
            pl.col("summary_llm"),
        ],
        separator=" ",
    ).alias("tit_summary"),
)

In [6]:
news.head(3)

index,topic,date_extract,date_article,content,portal,link,link_sim_score,title,summary,summary_llm,summary_sim_score,authors,state,city,content_hash,content_nchar,tit_summary
i64,str,str,str,str,str,str,f64,str,str,str,f64,str,str,str,u64,i64,str
1,"""narcotráfico""","""2023-08-16""","""2023-08-16""","""Seguinos Por L…","""https://www.ar…","""https://www.la…",0.561929,"""Condenaron a s…","""Brian está det…","""Seis personas …",0.5241,"""n-a""","""Santa Fe""","""Rosario""",9259959111029284149,4543,"""Santa Fe Rosar…"
2,"""narcotráfico""","""2023-08-16""","""2023-08-16""","""Escuadrón 1 “R…","""https://www.ar…","""http://www.dia…",0.476911,"""Encuentran 32 …","""na""","""La Fiscalía Fe…",0.5945,"""Diario El Oran…","""Salta""","""Orán""",6046667906668190206,707,"""Salta Orán Enc…"
3,"""narcotráfico""","""2023-08-16""","""2023-08-16""",""". Compartir en…","""https://www.ar…","""https://www.ar…",0.526558,"""Prefectura sec…","""Efectivos de l…","""La Prefectura …",0.5717,"""n-a""","""Argentina""","""Argentina""",12244461015022324118,657,"""Argentina Arge…"


## Embedings

In [7]:
text  = news['tit_summary'].to_list()[0]
text

'Santa Fe Rosario Condenaron a seis personas por venta de drogas, entre ellos un sobrino del "Rengo" Insaurralde Seis personas condenadas por el ‘Clan Insaurralde’ por tráfico y tenencia de estupefacientes. La jueza Elena Beatriz Dilario, miembro del Tribunal Oral Federal 3, homologó el acuerdo al que llegaron las partes'

In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(text)

In [9]:
len(embeddings)

384

In [10]:
type(embeddings)

numpy.ndarray

In [11]:
def process_dataframe(df):
    # Load the transformer model
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    # Function to apply the model to each element in 'tit_summary'
    def embed_text(summary):
        try:
            embeddings = model.encode(summary)
            return embeddings.tolist()
        except Exception as e:
            print(f"An error occurred while processing text: {summary}. Error: {str(e)}")
            return None # or return a default value if desired

    # Create a new column 'tit_sum_embed' with the embeddings of 'tit_summary'
    embeddings_col = df['tit_summary'].apply(embed_text, return_dtype=pl.Object)
    new_df = df.with_columns(embeddings_col.rename('tit_sum_embed'))

    return new_df


In [12]:
newsembed = process_dataframe(news)

In [13]:
newsembed['tit_sum_embed'].to_list()[0]

[0.029884234070777893,
 0.005878916010260582,
 -0.017692668363451958,
 -0.018943684175610542,
 -0.04433497413992882,
 -0.0014377949992194772,
 0.00997220166027546,
 0.032472047954797745,
 -0.008146866224706173,
 0.02436770685017109,
 0.07850281149148941,
 -0.046315889805555344,
 -0.1271422952413559,
 -0.010990813374519348,
 0.046851493418216705,
 -0.060365259647369385,
 -0.05267992615699768,
 0.08479571342468262,
 0.023379741236567497,
 0.06482059508562088,
 0.08752969652414322,
 -0.011996577493846416,
 -0.0478527769446373,
 0.08046647161245346,
 -0.05255422741174698,
 0.003865650622174144,
 -0.04024414345622063,
 -0.058590494096279144,
 -0.020242998376488686,
 -0.07578916847705841,
 -0.06450728327035904,
 0.06244717165827751,
 0.10415099561214447,
 -0.026164095848798752,
 0.004968362394720316,
 -0.00309031642973423,
 0.08883163332939148,
 -0.029620938003063202,
 -0.07091233134269714,
 8.030291792238131e-05,
 -0.12611861526966095,
 -0.0033292986918240786,
 -0.008059102110564709,
 -0.03

In [14]:
newsembed = (
    newsembed.with_columns([
        pl.col("tit_sum_embed").cast(pl.Utf8)
    ])    
)

ComputeError: cannot cast 'Object' type

In [25]:
newsembed.glimpse()

Rows: 44
Columns: 19
$ index                <i64> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
$ topic                <str> narcotráfico, narcotráfico, narcotráfico, narcotráfico, narcotráfico, narcotráfico, narcotráfico, narcotráfico, narcotráfico, narcotráfico
$ date_extract         <str> 2023-08-16, 2023-08-16, 2023-08-16, 2023-08-16, 2023-08-16, 2023-08-16, 2023-08-16, 2023-08-16, 2023-08-16, 2023-08-16
$ date_article         <str> 2023-08-16, 2023-08-16, 2023-08-16, 2023-08-14, 2023-08-16, 2023-07-22, 2023-08-16, 2023-08-16, 2023-08-16, 2023-08-16
$ content              <str> Seguinos Por Leo Graciarena La jueza Elena Beatriz Dilario, miembro del Tribunal Oral Federal 3, homologó el abreviado. Seis personas sindicadas como integrantes del “Clan Insaurralde” fueron condenadas por el Tribunal Oral Federal 3 en un proceso abreviado como responsables, en diferentes grados, del delito de tráfico y tenencia de estupefacientes con fines de comercialización agravada por la intervención de tres o más per

In [27]:
temp = newsembed.to_arrow()

In [28]:
temp

pyarrow.Table
index: int64
topic: large_string
date_extract: large_string
date_article: large_string
content: large_string
portal: large_string
link: large_string
link_sim_score: double
title: large_string
summary: large_string
summary_llm: large_string
summary_sim_score: double
authors: large_string
state: large_string
city: large_string
content_hash: uint64
content_nchar: int64
tit_summary: large_string
tit_sum_embed: fixed_size_binary[8]
----
index: [[1,2,3,4,5,...,40,41,42,43,44]]
topic: [["narcotráfico","narcotráfico","narcotráfico","narcotráfico","narcotráfico",...,"narcotráfico","narcotráfico","narcotráfico","narcotráfico","narcotráfico"]]
date_extract: [["2023-08-16","2023-08-16","2023-08-16","2023-08-16","2023-08-16",...,"2023-08-16","2023-08-16","2023-08-16","2023-08-16","2023-08-16"]]
date_article: [["2023-08-16","2023-08-16","2023-08-16","2023-08-14","2023-08-16",...,"2023-08-14","2023-08-16","2023-08-14","2023-08-16","2023-08-16"]]
content: [["Seguinos Por Leo Graciarena L

# write to mongo

In [29]:
from pymongo import MongoClient
from datetime import datetime

In [30]:
client = MongoClient('mongodb://localhost:27017/')
db = client['wdocuments']
collection = db['newsdata'] # export to mongo collection news

In [31]:
collection.insert_many(temp)

TypeError: documents must be a non-empty list